# Модуль Б
---
Участник 2

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os
import glob
from shapely.geometry import box
from rasterio.mask import mask
import folium
from shapely.geometry import Polygon
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans, AgglomerativeClustering, MiniBatchKMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

In [347]:
# Загружаем необходимые слои
etalons = gpd.read_file('Data/reprojected-wgs84/wgs84-etalons.shp')
etalons_clearcuts = gpd.read_file('Data/reprojected-wgs84/wgs84-etalons-clearcuts.shp')

In [5]:
etalons

,Kv,Sknr,Kf1,Mr1,Amz1,H1,D1,Skal1,Tur1h1,Kf2,...,Skal4,Tur1h4,Kf5,Mr5,Amz5,H5,D5,Skal5,Tur1h5,geometry
0,1,3,0,000000,0,0.0,0,0.0,0,0,...,0.0,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((64.85809 56.26744, 64.85822 56.26739..."
1,1,4,8,Б,65,23.0,26,0.6,170,2,...,0.0,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((64.85459 56.26687, 64.85505 56.26742..."
2,1,5,0,000000,0,0.0,0,0.0,0,0,...,0.0,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((64.86062 56.26735, 64.86188 56.26745..."
3,1,6,8,С,85,27.0,36,0.7,360,2,...,0.0,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((64.86372 56.26719, 64.86444 56.26729..."
4,1,7,0,000000,0,0.0,0,0.0,0,0,...,0.0,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((64.86595 56.26734, 64.86599 56.26665..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4638,124,21,10,Б,45,14.0,14,0.5,60,0,...,0.0,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((65.00049 56.14559, 65.00041 56.14523..."
4639,124,31,8,Б,60,23.0,24,0.6,170,1,...,0.0,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((65.00039 56.14514, 65.00015 56.144, ..."
4640,124,36,9,Б,45,19.0,18,0.8,170,1,...,0.0,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((65.0001 56.14377, 64.99963 56.14152,..."
4641,124,41,6,С,35,13.0,14,0.8,120,4,...,0.0,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((64.99961 56.14142, 64.99949 56.14075..."


## 1. Анализ и визуализация данных. Определение породного состава леса


#### Мой подход состоит в том, чтобы добавить атрибут с размерами участков и рассчитать распространение для каждой породы дерева

In [19]:
# Создаём новый атрибут
etalons = etalons.to_crs(epsg=3857)
etalons['plot_size'] = etalons['geometry'].area

In [21]:
etalons

,Kv,Sknr,Kf1,Mr1,Amz1,H1,D1,Skal1,Tur1h1,Kf2,...,Tur1h4,Kf5,Mr5,Amz5,H5,D5,Skal5,Tur1h5,geometry,plot_size
0,1,3,0,000000,0,0.0,0,0.0,0,0,...,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((7219969.128 7611841.601, 7219983.902...",15098.595977
1,1,4,8,Б,65,23.0,26,0.6,170,2,...,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((7219579.402 7611726.264, 7219631.294...",87624.024694
2,1,5,0,000000,0,0.0,0,0.0,0,0,...,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((7220251.214 7611823.377, 7220391.625...",6493.891059
3,1,6,8,С,85,27.0,36,0.7,360,2,...,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((7220596.688 7611789.932, 7220676.774...",89617.431197
4,1,7,0,000000,0,0.0,0,0.0,0,0,...,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((7220844.006 7611819.664, 7220848.711...",10545.511191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4638,124,21,10,Б,45,14.0,14,0.5,60,0,...,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((7235821.964 7587452.583, 7235812.971...",1632.102863
4639,124,31,8,Б,60,23.0,24,0.6,170,1,...,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((7235810.569 7587363.503, 7235783.293...",4584.537469
4640,124,36,9,Б,45,19.0,18,0.8,170,1,...,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((7235777.874 7587089.508, 7235725.73 ...",7958.676364
4641,124,41,6,С,35,13.0,14,0.8,120,4,...,0,0,000000,0,0.0,0,0.0,0,"POLYGON ((7235723.246 7586619.144, 7235710.136...",2344.001132


##### Как мы видим, у нас получился новый атрибут plot_size. Он измеряется в квадратных метрах

In [29]:
etalons['plot_size'].sum()

552810141.6072459

In [33]:
species_areas = etalons.groupby('Mr1')['plot_size'].sum()
total_area = species_areas.sum()

# Процент распределения
species_percentages = (species_areas / total_area) * 100

# Таблица с результатами
distribution_table = species_percentages.to_frame('percentage').reset_index()

print(distribution_table)

      Mr1  percentage
0  000000   11.815898
1       Б   30.189394
2       Е    0.444439
3       Л    0.143557
4      ОС    0.750558
5       С   56.656155


| Сокращение | Расшифровка                                        | % распространения |
|------------|----------------------------------------------------|-------------------|
| C          | Сосна обыкновенная                                 | 56.656155         |
| Е          | Ель европейская                                    | 0.444439          |
| Б          | Берёза повислая                                    | 30.189394         |
| ОС         | Осина                                              | 0.750558          |
| Л          | Листвинница сибирская                              | 0.143557          |
| 000000     | Не покрытые древесно-кустарниковой растительностью | 11.815898         |      |ю |   |

Метрики по ареалам

Возьмём Kv как ареал

In [349]:
grouped_etalons = etalons.groupby('Kv')

In [351]:
# Средний возраст леса
avg_age_by_kv = grouped_etalons.apply(lambda x: x[['Amz1', 'Amz2', 'Amz3', 'Amz4', 'Amz5']].stack().replace(0, np.nan).mean(skipna=True))
avg_age_df = avg_age_by_kv.reset_index()
avg_age_df.columns = ['Kv', 'avg_age']
avg_age_df

,Kv,avg_age
0,1,51.034483
1,2,48.189189
2,3,54.620690
3,4,60.661017
4,5,54.080000
...,...,...
144,145,64.000000
145,146,70.631579
146,147,56.294118
147,153,75.000000


In [339]:
# Средний запас древесины
avg_tur_by_kv = grouped_etalons.apply(lambda x: x[['Tur1h1', 'Tur1h2', 'Tur1h3', 'Tur1h4', 'Tur1h5']].stack().replace(0, np.nan).mean(skipna=True))
avg_tur_df = avg_tur_by_kv.reset_index()
avg_tur_df.columns = ['Kv', 'avg_tur']
avg_tur_df

,Kv,avg_tur
0,1,153.571429
1,2,161.379310
2,3,177.586207
3,4,225.277778
4,5,250.303030
...,...,...
143,144,255.909091
144,145,294.000000
145,146,296.000000
146,147,219.166667


In [76]:
# Средний запас древесины
avg_wood_stock = etalons[['Tur1h1', 'Tur1h2', 'Tur1h3', 'Tur1h4', 'Tur1h5']].stack().mean()
print(avg_wood_stock)

35.96338574197717


In [78]:
# Распределение в процентах зон
zone_distribution = etalons[['Mr1', 'Mr2', 'Mr3', 'Mr4', 'Mr5']].stack().value_counts(normalize=True) * 100
print(zone_distribution)

000000    62.476847
С         16.403188
Б         16.286883
ОС         4.415249
Е          0.379065
Л          0.025845
ИВ         0.012923
Name: proportion, dtype: float64


In [353]:
# Распределение по ареалам 

distribution_by_kv = grouped_etalons.apply(lambda x: x[['Mr1', 'Mr2', 'Mr3', 'Mr4', 'Mr5']].stack().value_counts(normalize=True) * 100)
distribution_df = distribution_by_kv.reset_index()
distribution_df.columns = ['Kv', 'Mr', 'proportion']
distribution_pivot = distribution_df.pivot_table(index='Kv', columns='Mr', values='proportion')
distribution_pivot = distribution_pivot[["000000", "Б", "Е", "Л", "ОС", "С"]]
distribution_pivot

Mr,000000,Б,Е,Л,ОС,С
Kv,,,,,,
1,69.285714,17.857143,NaN,NaN,2.142857,10.714286
2,62.857143,15.428571,NaN,NaN,5.714286,16.000000
3,52.500000,18.125000,NaN,NaN,6.250000,23.125000
4,60.000000,17.435897,NaN,NaN,3.589744,18.974359
5,56.315789,17.368421,1.578947,NaN,7.894737,15.789474
...,...,...,...,...,...,...
145,72.727273,13.636364,NaN,NaN,0.909091,12.727273
146,66.666667,14.444444,NaN,NaN,NaN,18.888889
147,65.714286,17.142857,NaN,NaN,NaN,17.142857


In [330]:
# Топ-10 с наибольшим запасом леса
df_sorted = etalons.sort_values(by='Kf1', ascending=False)
df_sorted.head(10)

,Kv,Sknr,Kf1,Mr1,Amz1,H1,D1,Skal1,Tur1h1,Kf2,...,Tur1h4,Kf5,Mr5,Amz5,H5,D5,Skal5,Tur1h5,geometry,Cluster
2322,71,6,10,3,55,20.0,18,0.7,160,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.97001 56.21719, 64.9707 56.21731,...",4
2924,91,13,10,1,5,0.9,0,0.8,0,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.81991 56.20176, 64.81997 56.20256...",1
2907,90,22,10,3,35,13.0,12,0.7,80,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((65.0072 56.19846, 65.00765 56.19821,...",4
2908,90,23,10,3,3,1.5,2,0.4,5,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((65.01145 56.19819, 65.01071 56.19816...",1
2909,90,24,10,3,25,10.0,8,0.5,40,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((65.01125 56.19726, 65.00857 56.19742...",1
2916,91,5,10,1,75,26.0,26,0.8,390,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.82184 56.20652, 64.82182 56.20597...",0
2917,91,6,10,1,110,26.0,32,0.7,340,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.81351 56.20688, 64.81549 56.20677...",0
1167,38,23,10,3,45,20.0,20,0.9,210,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.94814 56.23512, 64.94848 56.23516...",2
1166,38,22,10,3,15,8.0,10,0.8,40,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.95326 56.23632, 64.95354 56.23615...",1
2918,91,7,10,3,45,18.0,16,0.7,140,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.81292 56.20593, 64.81412 56.20599...",4


In [332]:
# Топ-10 с наибольшим запасом леса
df_sorted = etalons.sort_values(by=['Kf1', 'Kf2', 'Kf3', 'Kf4', 'Kf5'], ascending=False)
df_sorted.head(10)

,Kv,Sknr,Kf1,Mr1,Amz1,H1,D1,Skal1,Tur1h1,Kf2,...,Tur1h4,Kf5,Mr5,Amz5,H5,D5,Skal5,Tur1h5,geometry,Cluster
19,1,22,10,3,15,8.0,6,0.8,40,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.86077 56.26144, 64.86188 56.26152...",1
156,5,23,10,3,30,16.0,16,0.6,100,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.91961 56.25745, 64.92007 56.25727...",4
244,8,18,10,1,46,20.0,24,0.7,250,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.97369 56.2508, 64.97328 56.24966,...",2
251,8,25,10,1,15,5.0,6,0.8,50,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.97964 56.2502, 64.98201 56.24993,...",1
253,8,27,10,1,15,6.0,10,0.8,60,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.98466 56.24997, 64.98543 56.24975...",4
424,16,1,10,3,3,1.5,0,0.4,5,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.84726 56.2582, 64.84729 56.25823,...",1
456,16,33,10,3,10,5.0,4,0.4,10,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.83854 56.24832, 64.8397 56.24966,...",1
464,16,41,10,1,7,0.5,0,0.6,0,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.84422 56.24695, 64.84431 56.24669...",1
750,24,5,10,1,7,1.0,0,0.8,0,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.99471 56.24604, 64.99512 56.24629...",1
831,26,25,10,3,5,2.0,2,0.4,5,10,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((65.02399 56.23733, 65.02446 56.23745...",1


In [124]:
# Можно вывести также по размеру участка 
df_sorted = etalons.sort_values(by='plot_size', ascending=False)
df_sorted.head(10)

,Kv,Sknr,Kf1,Mr1,Amz1,H1,D1,Skal1,Tur1h1,Kf2,...,Kf5,Mr5,Amz5,H5,D5,Skal5,Tur1h5,geometry,plot_size,Total_Wood_Stock
1412,46,1,0,000000,0,0.0,0,0.0,0,0,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7247454.054 7602541.581, 7246969.021...",3.676172e+06,0
2525,78,1,0,000000,0,0.0,0,0.0,0,0,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7247196.427 7600392.202, 7247191.505...",3.369834e+06,0
906,30,1,0,000000,0,0.0,0,0.0,0,0,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7247703.573 7604432.681, 7246684.386...",2.885228e+06,0
1995,62,4,0,000000,0,0.0,0,0.0,0,0,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7247409.504 7602144.489, 7247389.247...",2.661252e+06,0
4253,137,21,10,Б,45,19.0,16,0.8,170,0,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7228294.244 7589510.71, 7228296.267 ...",2.018455e+06,180
423,15,17,0,000000,0,0.0,0,0.0,0,0,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7247892.151 7606044.09, 7245842.528 ...",1.929508e+06,0
3168,99,1,9,С,75,26.0,26,0.8,390,1,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7229240.658 7598010.416, 7229261.535...",1.918030e+06,390
3620,116,17,0,000000,0,0.0,0,0.0,0,0,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7219989.166 7593281.97, 7219754.47 7...",1.860770e+06,0
4495,146,1,5,С,95,26.0,28,0.9,390,4,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7232825.377 7586597.981, 7232847.422...",1.849010e+06,390
1399,45,7,0,000000,0,0.0,0,0.0,0,0,...,0,000000,0,0.0,0,0.0,0,"POLYGON ((7243761.678 7604133.648, 7243872.607...",1.840235e+06,0


## 2. Определение ареалов с зонами вырубки леса

In [129]:
etalons_clearcuts

,код,тип_к?,площа,код_с?,код_л?,код_?,код_?_1,субъе,лесни,уч_ле?,...,ущерб,наруш,вид_о?,резул,номе_2,дата_4,приме,приро,objectid,geometry
0,43100020101080201,Граница лесосеки по материалам отвода,3.8,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12334,"POLYGON ((64.89589 56.19062, 64.89668 56.19127..."
1,43100020101090101,Граница лесосеки по материалам отвода,3.5,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12336,"POLYGON ((64.91431 56.189, 64.91417 56.18794, ..."
2,43100020101180201,Граница лесосеки по материалам отвода,1.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12337,"POLYGON ((64.88695 56.18672, 64.88767 56.18667..."
3,43100020101180301,Граница лесосеки по материалам отвода,3.2,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12338,"POLYGON ((64.90145 56.18564, 64.90368 56.18544..."
4,43100020101240101,Граница лесосеки по материалам отвода,9.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12321,"POLYGON ((64.98932 56.17592, 64.99024 56.17611..."
5,43100020101240201,Граница лесосеки по материалам отвода,3.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12323,"POLYGON ((64.99039 56.1699, 64.99104 56.1702, ..."
6,43100020101260101,Граница лесосеки по материалам отвода,2.6,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12348,"POLYGON ((64.89405 56.17, 64.89446 56.17028, 6..."
7,43100020101320101,Граница лесосеки по материалам отвода,5.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12329,"POLYGON ((64.99363 56.16606, 64.99436 56.16633..."
8,43100020101320201,Граница лесосеки по материалам отвода,2.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12330,"POLYGON ((64.99014 56.16515, 64.99124 56.16541..."
9,43100020101320301,Граница лесосеки по материалам отвода,2.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,0.0,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12332,"POLYGON ((64.98687 56.1637, 64.98773 56.16341,..."


In [131]:
etalons_clearcuts.columns

Index(['код', 'тип_к?', 'площа', 'код_с?', 'код_л?', 'код_?', 'код_?_1',
       'субъе', 'лесни', 'уч_ле?', 'урочи', 'кварт', 'выдел', 'лесос',
       'код_д?', 'код_к?', 'вид_и?', 'вид_п?', 'аренд', 'ИНН', 'докум',
       'дата_?', 'номер', 'дата_1', 'номе_1', 'дата_2', 'ЕГАИС', 'дата_3',
       'соста', 'запас', 'общий', 'несоо', 'защит', 'ОЗУ', 'ущерб', 'наруш',
       'вид_о?', 'резул', 'номе_2', 'дата_4', 'приме', 'приро', 'objectid',
       'geometry'],
      dtype='object')

In [133]:
# Создаём новый атрибут для определения площади вырубки
etalons_clearcuts = etalons_clearcuts.to_crs(epsg=3857)
etalons_clearcuts['plot_size'] = etalons_clearcuts['geometry'].area

In [135]:
etalons_clearcuts

,код,тип_к?,площа,код_с?,код_л?,код_?,код_?_1,субъе,лесни,уч_ле?,...,наруш,вид_о?,резул,номе_2,дата_4,приме,приро,objectid,geometry,plot_size
0,43100020101080201,Граница лесосеки по материалам отвода,3.8,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12334,"POLYGON ((7224177.652 7596457.071, 7224265.706...",120870.338841
1,43100020101090101,Граница лесосеки по материалам отвода,3.5,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12336,"POLYGON ((7226227.489 7596133.381, 7226212.795...",111708.557365
2,43100020101180201,Граница лесосеки по материалам отвода,1.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12337,"POLYGON ((7223182.456 7595676.878, 7223262.494...",43585.989201
3,43100020101180301,Граница лесосеки по материалам отвода,3.2,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12338,"POLYGON ((7224796.143 7595459.839, 7225044.72 ...",104109.356509
4,43100020101240101,Граница лесосеки по материалам отвода,9.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12321,"POLYGON ((7234577.564 7593516.158, 7234680.312...",302268.157744
5,43100020101240201,Граница лесосеки по материалам отвода,3.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12323,"POLYGON ((7234697.344 7592313.355, 7234769.59 ...",109577.885359
6,43100020101260101,Граница лесосеки по материалам отвода,2.6,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12348,"POLYGON ((7223972.602 7592333.35, 7224017.686 ...",84242.501069
7,43100020101320101,Граница лесосеки по материалам отвода,5.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12329,"POLYGON ((7235057.908 7591543.979, 7235139.06 ...",174844.870729
8,43100020101320201,Граница лесосеки по материалам отвода,2.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12330,"POLYGON ((7234669.069 7591362.443, 7234792.188...",77604.591711
9,43100020101320301,Граница лесосеки по материалам отвода,2.4,43,10,002,01,Курганская область,Шатровское,Бариновское,...,Без нарушения,Не установлен,Не проводилась,0,None,"Рубка на арендованном лесном участке, включенн...",None,12332,"POLYGON ((7234304.831 7591073.954, 7234401.345...",76544.086722


In [139]:
etalons_clearcuts['plot_size'].sum()

2757127.7342337095

Общая площадь вырубки составила 2 757 127 квадратных метров!

In [146]:
etalons_clearcuts['Total_Area'] = etalons_clearcuts['plot_size']
etalons_clearcuts_sorted = etalons_clearcuts.sort_values(by='Total_Area', ascending=False)

# Топ-10 больших участков с большой площадью выбурбки
top_10_clearcuts = etalons_clearcuts_sorted.head(10)
print("Топ-10 участков с наибольшей площадью вырубки:")
print(top_10_clearcuts[['Total_Area', 'geometry']])

Топ-10 участков с наибольшей площадью вырубки:
       Total_Area                                           geometry
15  391622.810566  POLYGON ((7232722.425 7605986.021, 7232768.288...
4   302268.157744  POLYGON ((7234577.564 7593516.158, 7234680.312...
20  249723.505162  POLYGON ((7233293.828 7599508.386, 7233321.435...
7   174844.870729  POLYGON ((7235057.908 7591543.979, 7235139.06 ...
11  151153.390786  POLYGON ((7228131.164 7587521.809, 7228160.441...
21  149117.351872  POLYGON ((7218726.337 7599336.869, 7218815.615...
16  137709.204185  POLYGON ((7236729.593 7605308.609, 7236787.59 ...
14  132972.791631  POLYGON ((7222846.939 7606578.155, 7222922.97 ...
0   120870.338841  POLYGON ((7224177.652 7596457.071, 7224265.706...
18  116265.213116  POLYGON ((7238170.735 7601403.738, 7238363.54 ...


In [172]:
top_10_clearcuts = top_10_clearcuts.to_crs(epsg=4326)

In [176]:
# Создадим карту через Folium
m = folium.Map(location=[64.966761, 56.287650], zoom_start=10, attributionControl=0)

for index, row in top_10_clearcuts.iterrows():
    polygon = Polygon(row['geometry'].exterior.coords)
    coordinates = [(x, y) for x, y in polygon.exterior.coords]
    folium.Polygon(coordinates).add_to(m)

m

In [178]:
# Сохраним карту
m.save('m.html')

## 3. Сегментация изображений - кластеризации ареалов леса по типам, возрасту,

## запасу древесины

In [233]:
df = etalons

In [237]:
tree_species_map = {
    'С': 1,  # Сосна обыкновенная
    'Е': 2,  # Ель европейская
    'Б': 3,  # Берёза повислая
    'ОС': 4,  # Осина
    'Л': 5,  # Лиственница сибирская
    '000000': 0  # Не покрытые древесно-кустарниковой растительностью
}

# Replace the abbreviations with numerical values
df['Mr1'] = df['Mr1'].map(tree_species_map)
df['Mr2'] = df['Mr2'].map(tree_species_map)
df['Mr3'] = df['Mr3'].map(tree_species_map)
df['Mr4'] = df['Mr4'].map(tree_species_map)
df['Mr5'] = df['Mr5'].map(tree_species_map)

In [243]:
df.dropna(inplace = True)

### Рассмотрим алгоритмы кластеризации

#### KMeans
Функция k-средних разделяет точки данных, имеющие сходства, на кластеры. Кластеры становятся более компактными и дифференцированными по мере применения алгоритма k-средних для настраиваемого числа итераций.

#### Agglomerative (алгомеративная)

Относится к семейству алгоритмов кластеризации, в основе которых лежат одинаковые принципы: алгоритм начинает свою работу с того, что каждую точку данных заносит в свой собственный кластер и по мере выполнения объединяет два наиболее схожих между собой кластера до тех пор, пока не будет удовлетворен определенный критерии остановки. Критерий остановки, реализованный в scikit-learn - это количество кластеров, поэтому схожие между собой кластеры объединяются до тех пор, пока не останется заданное число кластеров. Есть несколько критериев связи (linkage), которые задают точный способ измерения "наиболее схожего кластера". В основе этих критериев лежит расстояние между двумя существующими кластерами.

#### MiniBatch KMeans

Вариант KMeans алгоритма, который использует мини-пакеты для сокращения времени вычислений, но при этом пытается оптимизировать ту же целевую функцию. Мини-пакеты — это подмножества входных данных, которые выбираются случайным образом на каждой итерации обучения.

### Рассмотрим метрики

#### Коэффициент силуэта

Коэффициент силуэта показывает, насколько каждый объект «похож» на другие объекты в том кластере, в который он был распределен в процессе кластеризации, и «не похож» на объекты из других кластеров. Значение коэффициента силуэта находится в диапазоне от −1 до 1. Значение a(o) отражает компактность кластера, к которому принадлежит o. Чем меньше значение, тем компактнее кластер. Значение b(o) отражает степень, в которой o отделен от других кластеров.

#### Индекс Калински-Харабаша
Критерий Калински-Харабаша иногда называют критерием отношения дисперсии (VRC) . Четко определенные кластеры имеют большую межкластерную дисперсию и маленькую внутрикластерную дисперсию. Оптимальное количество кластеров соответствует решению с наибольшим значением индекса Калински-Харабаша. Компактность основана на расстоянии от точек кластера до их центроидов, а разделимость - на расстоянии от центроид кластеров до глобального центроида. Должен возрастать.

#### Индекс Дэвиcа-Болдуина

Это, возможно, одна из самых используемых мер оценки качества кластеризации.
Она вычисляет компактность как расстояние от объектов кластера до их центроидов, а отделимость - как расстояние между центроидами
.

#### Показания нескольких метрик лучше всего показывает насколько хорошо прошлая кластеризация. Одну метрику использовать не имеет смысла, поэтому я выбрал 3 самые основные метрики

In [257]:
# Необходимые переменные
X = df[['Amz1', 'H1', 'D1', 'Skal1', 'Tur1h1', 'Kf1', 'Mr1', 'Amz2', 'H2', 'D2', 'Skal2', 'Tur1h2', 'Kf2', 'Mr2', 'Amz3', 'H3', 'D3', 'Skal3', 'Tur1h3', 'Kf3', 'Mr3', 'Amz4', 'H4', 'D4', 'Skal4', 'Tur1h4', 'Kf4', 'Mr4', 'Amz5', 'H5', 'D5', 'Skal5', 'Tur1h5', 'Kf5', 'Mr5']]

# Обучаем
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans_labels = kmeans.fit_predict(X)

agglo = AgglomerativeClustering(n_clusters=5, linkage='ward')
agglo_labels = agglo.fit_predict(X)

mbkmeans = MiniBatchKMeans(n_clusters=5, batch_size=100, random_state=42)
mbkmeans_labels = mbkmeans.fit_predict(X)

# Cчитаем метрики
kmeans_silhouette = silhouette_score(X, kmeans_labels)
kmeans_calinski = calinski_harabasz_score(X, kmeans_labels)
kmeans_davies = davies_bouldin_score(X, kmeans_labels)

agglo_silhouette = silhouette_score(X, agglo_labels)
agglo_calinski = calinski_harabasz_score(X, agglo_labels)
agglo_davies = davies_bouldin_score(X, agglo_labels)

mbkmeans_silhouette = silhouette_score(X, mbkmeans_labels)
mbkmeans_calinski = calinski_harabasz_score(X, mbkmeans_labels)
mbkmeans_davies = davies_bouldin_score(X, mbkmeans_labels)

print("K-Means:")
print("Silhouette Coefficient:", kmeans_silhouette)
print("Calinski-Harabasz Index:", kmeans_calinski)
print("Davies-Bouldin Index:", kmeans_davies)

print("\nAgglomerativeClustering:")
print("Silhouette Coefficient:", agglo_silhouette)
print("Calinski-Harabasz Index:", agglo_calinski)
print("Davies-Bouldin Index:", agglo_davies)

print("\nMini-Batch K-Means:")
print("Silhouette Coefficient:", mbkmeans_silhouette)
print("Calinski-Harabasz Index:", mbkmeans_calinski)
print("Davies-Bouldin Index:", mbkmeans_davies)

K-Means:
Silhouette Coefficient: 0.4007772441625512
Calinski-Harabasz Index: 8136.0922013300515
Davies-Bouldin Index: 1.0311184716475363

AgglomerativeClustering:
Silhouette Coefficient: 0.3588372703162703
Calinski-Harabasz Index: 6442.187544670838
Davies-Bouldin Index: 0.8630137215782516

Mini-Batch K-Means:
Silhouette Coefficient: 0.4411995227003478
Calinski-Harabasz Index: 7537.212970492857
Davies-Bouldin Index: 1.0002288276898597


Судя по метрикам оценки, кажется, что Mini Batch KMeans работает лучше всего с более высоким коэффициентом силуэта и индексом Калински-Харабаша по сравнению с KMeans и агломеративной кластеризацией. Индекс Дэвиса-Булдина также относительно низок для Mini Batch KMeans, что указывает на то, что кластеры хорошо разделены и компактны.

In [276]:
# Кластеризация
mbkmeans = MiniBatchKMeans(n_clusters=5, batch_size=100, random_state=42)
mbkmeans_labels = mbkmeans.fit_predict(X)

# Новый атрибут с кластерами
df['Cluster'] = mbkmeans_labels

In [280]:
# Атрибуты в разрезе кластера
for cluster in range(5):
    cluster_data = df[df['Cluster'] == cluster]
    print(f"Cluster {cluster}:")
    print("Amz1:", cluster_data['Amz1'].mean())
    print("H1:", cluster_data['H1'].mean())
    print("D1:", cluster_data['D1'].mean())
    print("Skal1:", cluster_data['Skal1'].mean())
    print("Tur1h1:", cluster_data['Tur1h1'].mean())
    print("Kf1:", cluster_data['Kf1'].mean())
    print("Mr1:", cluster_data['Mr1'].mean())
    print()

Cluster 0:
Amz1: 73.34932126696833
H1: 24.716742081447965
D1: 28.941176470588236
Skal1: 0.7749321266968326
Tur1h1: 354.9864253393665
Kf1: 8.455203619909502
Mr1: 1.0235294117647058

Cluster 1:
Amz1: 8.802931596091206
H1: 3.323127035830619
D1: 3.3566775244299674
Skal1: 0.22475570032573286
Tur1h1: 11.44543973941368
Kf1: 4.076547231270358
Mr1: 1.0643322475570032

Cluster 2:
Amz1: 56.86159486709441
H1: 21.664527956003667
D1: 23.501374885426216
Skal1: 0.7307974335472044
Tur1h1: 229.64252978918424
Kf1: 8.065994500458295
Mr1: 2.0238313473877176

Cluster 3:
Amz1: 58.69260700389105
H1: 21.94941634241245
D1: 23.642023346303503
Skal1: 0.7447470817120623
Tur1h1: 241.24513618677042
Kf1: 7.190661478599222
Mr1: 2.0700389105058368

Cluster 4:
Amz1: 41.657262277951936
H1: 15.2675026123302
D1: 15.661442006269592
Skal1: 0.6949843260188087
Tur1h1: 117.84743991640543
Kf1: 8.595611285266457
Mr1: 2.4012539184952977



Кластер 0: Зрелый лес. Этот кластер имеет самые высокие значения для Amz1, H1, D1 и Tur1h1, что указывает на зрелый лес с высокими деревьями и густым пологом.

Кластер 1: Молодой лес. Этот кластер имеет относительно низкие значения Amz1, H1, D1 и Tur1h1, что предполагает более молодой лес с меньшими деревьями и менее густым пологом.

Кластер 2: Средневозрастной лес. Этот кластер имеет промежуточные значения для Amz1, H1, D1 и Tur1h1, что указывает на средневозрастной лес с деревьями умеренной высоты и густоты.

Кластер 3: Плотная заросль. Этот кластер имеет высокие значения Skal1 и Tur1h1, что указывает на густую заросль с высокой плотностью деревьев и подлеска.

Кластер 4: Редкий лесной массив. Этот кластер имеет относительно низкие значения для Amz1, H1, D1 и Tur1h1, что указывает на открытый лесной массив с более редким древесным покровом и более открытым пологом.

In [290]:
df

,Kv,Sknr,Kf1,Mr1,Amz1,H1,D1,Skal1,Tur1h1,Kf2,...,Tur1h4,Kf5,Mr5,Amz5,H5,D5,Skal5,Tur1h5,geometry,Cluster
0,1,3,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.85809 56.26744, 64.85822 56.26739...",1
1,1,4,8,3,65,23.0,26,0.6,170,2,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.85459 56.26687, 64.85505 56.26742...",2
2,1,5,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.86062 56.26735, 64.86188 56.26745...",1
3,1,6,8,1,85,27.0,36,0.7,360,2,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.86372 56.26719, 64.86444 56.26729...",0
4,1,7,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.86595 56.26734, 64.86599 56.26665...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4638,124,21,10,3,45,14.0,14,0.5,60,0,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((65.00049 56.14559, 65.00041 56.14523...",4
4639,124,31,8,3,60,23.0,24,0.6,170,1,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((65.00039 56.14514, 65.00015 56.144, ...",3
4640,124,36,9,3,45,19.0,18,0.8,170,1,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((65.0001 56.14377, 64.99963 56.14152,...",4
4641,124,41,6,1,35,13.0,14,0.8,120,4,...,0,0,0,0,0.0,0,0.0,0,"POLYGON ((64.99961 56.14142, 64.99949 56.14075...",4


# 4. Отчёт

## 1. Анализ и визуализация данных. Определение породного состава леса

Для этого был взят датасет с шейпами, был создан новый атрибут с площадью полигонов. В итоге для каждой породы дерева было выведено распространенение в виде таблицы.
Кроме этого, была рассчитана площадь породного состава леса (по каждой породе), средний возраст и запас древесины леса, распределение в процентах зон, а также топ-10 участков с наибольшим запасом леса. 

## 2. Определение ареалов с зонами вырубки леса

Были определены ареалы с зонами проведения рубок леса за промежуток времени с 2021 по 2024 годы, был также создан атрибути с площадью вырубки. Общая площадь вырубки составила более 2 млн квадратных метров! Кроме этого, определены зоны с наибольшей площадью вырубки. Данные зоны были отображены на карте с помощью библиотеки Folium.

## 3. Сегментация изображений - кластеризация ареалов леса по типам, возрасту, запасам древесины
Были рассмотрены 3 алгоритма кластеризации и три метрики. Анализ показал, что Mini Batch KMeans является наилучшим алгоритмом для нашей задачи. Создано 5 кластеров, каждый из них поименован. Дана характеристика кластерам. Сделаны выводы о качстве работы каждого из алгоритмов. Добавлен номер кластера как дополнительный атрибут в исходный массив данных.

## 4. Отчёт

Был создан данный отчёт = )

Кроме этого, хочу добавить, что кроме данного отчёта, я также предоставил файл с моей картой (пункт 2)

In [356]:
df.to_csv('newdata.csv', index = False)